# Sokoban Ontology Exploration
The file takes in the Sokoban ontology and explores the triples in the ontology.

In [1]:
import rdflib
from typing import List
from pprint import pprint

In [2]:
file_path: str = "../../data/blocksworld/plan-ontology-rdf-instances_blocksworld.owl"
g: rdflib.graph.Graph = rdflib.Graph().parse(file_path, format="xml")

In [3]:
print(len(g))

502


In [4]:
# Get all triples in the ontology
with open("./blocksworld_triples.txt", "w") as f:
    for s, p, o in g:
        f.write(f"{s}\t{p}\t{o}\n")

<br>

In [50]:
def get_actions_from_rdf(g: rdflib.Graph, domain: str) -> List[str]:
    """
    Get the actions from an RDF graph.
    @param g: The RDF graph.
    @param domain: The domain of the actions.
    @return: The actions from the RDF graph. Type: List[query.ResultRow]
    """
    to_return: List[rdflib.query.ResultRow] = []

    query = f"""
            PREFIX planning: <https://purl.org/ai4s/ontology/planning#>
    
            SELECT ?action
            WHERE {{
                planning:{domain} planning:hasAction ?action .
            }}
            """
    
    for r in g.query(query):
        to_return.append(r)
    
    to_return = [str(lit[0]).split("#")[-1] for lit in to_return]
    
    # Remove "-" in some words
    # to_return.extend([lit.replace("-", "") for lit in to_return if "-" in lit])
    return to_return

In [51]:
domain: str = "blocksworld"
actions: List[str] = get_actions_from_rdf(g, domain)
print(f"Actions in the {domain} domain:")
print(actions)

Actions in the blocksworld domain:
['pick-up', 'put-down', 'stack', 'unstack']


In [52]:
def get_parameters_from_rdf(g: rdflib.Graph, action: str) -> List[str]:
    """
    Get the actions from an RDF graph.
    @param g: The RDF graph.
    @param domain: The domain of the actions.
    @return: The actions from the RDF graph. Type: List[query.ResultRow]
    """
    to_return: List[rdflib.query.ResultRow] = []

    query = f"""
            PREFIX planning: <https://purl.org/ai4s/ontology/planning#>

            SELECT ?param
            WHERE {{
                planning:{action} planning:hasParameter ?param .
            }}
            """
    
    for r in g.query(query):
        to_return.append(r)
    
    to_return = [str(lit[0]).split("#")[-1] for lit in to_return]
    
    # Remove "-" in some words
    to_return.extend([lit.replace("-", "") for lit in to_return if "-" in lit])
    return to_return

In [53]:
# Get all parameters for the specified action
action = "pick-up"
parameters = get_parameters_from_rdf(g, action)

print(f"Parameters for {action}:")
print(parameters)

Parameters for pick-up:
['?x']


In [7]:
# Get all preconditions for the specified action
action = "pushup"
preconditions = rdf_utils.get_preconditions_from_rdf(g, action)

print(f"Preconditions for {action}:")
print(preconditions)

Preconditions for pushup:
['sokoban ?sokoban', 'crate ?crate', 'below ?x ?y', 'below ?y ?z', 'at ?sokoban ?x', 'at ?crate ?y', 'clear ?z']


In [8]:
# Get all effects for the specified action
action = "pushup"
effects = rdf_utils.get_effects_from_rdf(g, action)

print(f"Effects for {action}:")
print(effects)

Effects for pushup:
['at ?sokoban ?y', 'at ?crate ?z', 'clear ?x', 'not (at ?sokoban ?x)', 'not (at ?crate ?y)', 'not (clear ?y)', 'not (clear ?z)']


In [9]:
action_details = dict.fromkeys(actions)  # Dictionary to store all details for each action

for action in actions:
    # Get all preconditions & effects for the specified action
    # Store in the dictionary
    action_details[action] = (rdf_utils.get_preconditions_from_rdf(g, action), rdf_utils.get_effects_from_rdf(g, action))

In [10]:
pprint(action_details)

{'movedown': (['sokoban ?sokoban', 'at ?sokoban ?x', 'below ?y ?x', 'clear ?y'],
              ['at ?sokoban ?y',
               'clear ?x',
               'not (at ?sokoban ?x)',
               'not (clear ?y)']),
 'moveleft': (['sokoban ?sokoban',
               'at ?sokoban ?x',
               'leftOf ?y ?x',
               'clear ?y'],
              ['at ?sokoban ?y',
               'clear ?x',
               'not (at ?sokoban ?x)',
               'not (clear ?y)']),
 'moveright': (['sokoban ?sokoban',
                'at ?sokoban ?x',
                'leftOf ?x ?y',
                'clear ?y'],
               ['at ?sokoban ?y',
                'clear ?x',
                'not (at ?sokoban ?x)',
                'not (clear ?y)']),
 'moveup': (['sokoban ?sokoban', 'at ?sokoban ?x', 'below ?x ?y', 'clear ?y'],
            ['at ?sokoban ?y',
             'clear ?x',
             'not (at ?sokoban ?x)',
             'not (clear ?y)']),
 'pushdown': (['sokoban ?sokoban',
               